In [2]:
#!pip install simpletransformers

In [32]:
#!pip install torchvision
#!pip install tensorflow_text

In [1]:
import numpy as np
import pandas as pd
import warnings
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
import tldextract   # Accurately separates a URL's subdomain, domain, and public suffix
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import re
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import nltk 
import spacy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from wordcloud import WordCloud, ImageColorGenerator,STOPWORDS
import gensim
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords
import copy
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,LancasterStemmer
from sklearn.metrics import f1_score
from pprint import pprint
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 1000)

[nltk_data] Downloading package stopwords to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
pd.set_option('display.max_colwidth', 200)
#pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)

In [3]:
foldr_pol_real = 'input_dataset//fakenewsnet//politifact//real//'
foldr_pol_fake = 'input_dataset//fakenewsnet//politifact//fake//'
foldr_gos_real = 'input_dataset//fakenewsnet//gossipcop//real//'
foldr_gos_fake = 'input_dataset//fakenewsnet//gossipcop//fake//'

##### Function to extract or convert JSON content to News Content format in dataframe

In [6]:
def json_to_newscontent(datapath):
    dictlist = []
    cols = ['url','text','title','authors','num_images','domain']
    folders = glob.glob(datapath+'/*')
    for index, subdir in enumerate(folders):
        path_file = glob.glob(subdir+'/*')
        #check whether file path is valid or not
        if len(path_file) == 1:
            file = open(path_file[0]).read()
            jsondata = json.loads(file)
            thedict = {'url':jsondata['url'],'title':jsondata['title'],'text':jsondata['text'],
                   'num_images':len(jsondata['images']),'authors':str(jsondata['authors'])}
            extrt_url = tldextract.extract(jsondata['url'])
            thedict['domain'] = extrt_url.domain
            dictlist.append(thedict)
    df=pd.DataFrame(dictlist,columns=cols)
    return df

##### Converting seperate dataframes for Politfact fake and real and Gossip fake and real

In [7]:
import time

# get the start time
st = time.time()

df_pol_real = json_to_newscontent(foldr_pol_real)
df_pol_fake = json_to_newscontent(foldr_pol_fake)
df_gos_real = json_to_newscontent(foldr_gos_real)
df_gos_fake = json_to_newscontent(foldr_gos_fake)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 313.5928819179535 seconds


In [8]:
## Cleaned dataset
df_pol_real = df_pol_real.loc[df_pol_real["text"] != '' ]
df_pol_fake = df_pol_fake.loc[df_pol_fake["text"] != '' ]
df_gos_real = df_gos_real.loc[df_gos_real["text"] != '' ]
df_gos_fake = df_gos_fake.loc[df_gos_fake["text"] != '' ]

df_pol_real.reset_index(drop=True,inplace=True)
df_pol_fake.reset_index(drop=True,inplace=True)
df_gos_real.reset_index(drop=True,inplace=True)
df_gos_fake.reset_index(drop=True,inplace=True)

In [9]:
df_pol_real['Category'] = 'Politics' 
df_pol_fake['Category'] = 'Politics'
df_gos_real['Category'] = 'Gossips'
df_gos_fake['Category'] = 'Gossips'

##### Add response variable 'class' based on True=1 or Fake=0

In [10]:
df_pol_real['class'] = 0 #True
df_pol_fake['class'] = 1 #Fake
df_gos_real['class'] = 0
df_gos_fake['class'] = 1

##### Storing dataframe to csv file 

In [11]:
df_pol_real.to_csv('Updated//df_pol_real.csv', index=False)
df_pol_fake.to_csv('Updated//df_pol_fake.csv', index=False)
df_gos_real.to_csv('Updated//df_gos_real.csv', index=False)
df_gos_fake.to_csv('Updated//df_gos_fake.csv', index=False)

#### Creating dataframe from csv

In [13]:
# #creating a data frame
df_pol_real = pd.read_csv("Updated//df_pol_real.csv")
df_pol_fake = pd.read_csv("Updated//df_pol_fake.csv")
df_gos_real = pd.read_csv("Updated//df_gos_real.csv")
df_gos_fake = pd.read_csv("Updated//df_gos_fake.csv")

In [ ]:
# df_pol_real.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# df_pol_fake.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# df_gos_real.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# df_gos_fake.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [14]:
df_list = [df_pol_real,df_pol_fake,df_gos_real,df_gos_fake]

In [15]:
#combine df's into one
df = pd.concat(df_list).reset_index(drop=True)
total_rows = df.shape[0]

In [16]:
df['class'].value_counts() / df.shape[0]*100

0    75.193989
1    24.806011
Name: class, dtype: float64

##### combining title and text to full text column

In [17]:
df['fulltext']=df['title']+'.'+df['text']
#df2=df2.drop(columns=['url', 'text','title','authors','num_images','domain','Category','url_filter'], axis=1)
df=df.dropna()
df.shape

(20362, 9)

### Saving Fulldataset

In [3]:
#df.to_csv('Updated//full_dataset.csv', index=False)

### Loading the Dataset

In [2]:
#creating a data frame
df = pd.read_csv("Updated//full_dataset.csv")

In [3]:
filterd_df=df[['fulltext','class']]

In [4]:
filterd_df.head(2)

,fulltext,class
0,"Rick Santorum: By amending constitution, we can balance the budget for good.Rick Santorum: By amending constitution, we can balance the budget for good\n\nBy RICK SANTORUM\n\nLast summer, we saw a great groundswell of national support to get our fiscal house in order by cutting and capping federal spending and passing a balanced budget amendment to the Constitution. However, the President refused to lead and instead pushed the tough decisions to a bureaucratic Washington committee to solve the problem, and though it was named the 'Supercommittee' it couldn't even agree on cutting Post-It Notes from the office supply closet. Now, because of the inaction, we are faced with drastic defense cuts at a time when reports show a nuclear Iran has emerged as a clear and present danger.I supported the 'Cut, Cap, and Balance' movement, but the reality is that what we really need is 'Balance, Balance, Grow' - because without a balanced budget amendment, and a path to economic prosperity, we can...",0
1,"CNN's Don Lemon: Bill O'Reilly's Criticism Of Black Community ""Doesn't Go Far Enough"".Posted on July 27, 2013\n\nCNN's Don Lemon: Bill O'Reilly's Criticism Of Black Community ""Doesn't Go Far Enough""\n\nPlease enable Javascript to watch this video\n\nDON LEMON, CNN ANCHOR: I want to talk to you because we're going to take a break from the headlines to talk about something I've had on my mind for quite some time now. So much so that I felt compelled to bring back our segment where we hold politicians, leaders, and pundits accountable for what comes out of their mouths. It's time now, again, for No Talking Points.\n\n\n\nThe Trayvon Martin murder case got just about everybody talking about race, and not just specifically how it related to the case. It got some, many on the political right, wondering why the so-called liberal media wasn't talking about other problems in the black community.\n\n\n\n(BEGIN VIDEO CLIP)\n\n\n\nJUDY MILLER, FOX NEWS CONTRIBUTOR: Let's talk about race. Let's...",0


In [5]:
# Clean data using the built in cleaner in gensim
filterd_df['cleantext'] = filterd_df['fulltext'].apply(lambda x: gensim.utils.simple_preprocess(x))
filterd_df.head(1)

,fulltext,class,cleantext
0,"Rick Santorum: By amending constitution, we can balance the budget for good.Rick Santorum: By amending constitution, we can balance the budget for good\n\nBy RICK SANTORUM\n\nLast summer, we saw a great groundswell of national support to get our fiscal house in order by cutting and capping federal spending and passing a balanced budget amendment to the Constitution. However, the President refused to lead and instead pushed the tough decisions to a bureaucratic Washington committee to solve the problem, and though it was named the 'Supercommittee' it couldn't even agree on cutting Post-It Notes from the office supply closet. Now, because of the inaction, we are faced with drastic defense cuts at a time when reports show a nuclear Iran has emerged as a clear and present danger.I supported the 'Cut, Cap, and Balance' movement, but the reality is that what we really need is 'Balance, Balance, Grow' - because without a balanced budget amendment, and a path to economic prosperity, we can...",0,"[rick, santorum, by, amending, constitution, we, can, balance, the, budget, for, good, rick, santorum, by, amending, constitution, we, can, balance, the, budget, for, good, by, rick, santorum, last, summer, we, saw, great, groundswell, of, national, support, to, get, our, fiscal, house, in, order, by, cutting, and, capping, federal, spending, and, passing, balanced, budget, amendment, to, the, constitution, however, the, president, refused, to, lead, and, instead, pushed, the, tough, decisions, to, bureaucratic, washington, committee, to, solve, the, problem, and, though, it, was, named, the, supercommittee, it, couldn, even, agree, on, cutting, post, it, notes, from, the, office, supply, closet, now, because, ...]"


### Splitting dataset into train and test

In [6]:
X=filterd_df['cleantext']
y=filterd_df['class']

In [7]:
#https://medium.com/@dilip.voleti/classification-using-word2vec-b1d79d375381

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=0)

In [9]:
# Train the word2vec model from our own dataset
w2v_embedng_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [10]:
# We need to make aggregated sentence vectors to replace all words in the sentences with the learned word vectors, 
#then we need to replace each text message with the learned word vectors

words = set(w2v_embedng_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_embedng_model.wv[i] for i in m if i in words])
                         for m in X_train])
X_test_vect = np.array([np.array([w2v_embedng_model.wv[i] for i in n if i in words])
                         for n in X_test])

In [18]:
len(words)

63135

In [19]:
len(X_train_vect[0])

193

In [20]:
X_train_vect[0].size

19300

In [11]:
# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training

X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [21]:
# https://radimrehurek.com/gensim/models/word2vec.html
# https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview/part-3-more-fun-with-word-vectors

### Helper Functions

In [33]:
import timeit
from timeit import default_timer as timer
from datetime import timedelta
 


def metrics(X_test,y_test,clf):
    predictions=clf.predict(X_test)
    #predictions=(clf.predict_proba(X_test)[:,1] >= 0.3).astype(bool)
    print("confusion_matrix :")
    print(confusion_matrix(y_test,predictions))
    print("Accuracy Score :")
    print(accuracy_score(y_test, predictions))
    print("Classification Report :")
    print(classification_report(y_test, predictions))
    print("F1 score :")
    print(f1_score(y_test, predictions))
    print("ROC AUC Score")
    y_pred_proba = clf.predict_proba(X_test)
    print(roc_auc_score(y_test, y_pred_proba[:,1]) )
    print("------------------------------")

    
def model_comparison_table(X_test,y_test,classifier_list):
    dict_clf={}
    for clf_name,clf in classifier_list:
        predictions=clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)
        accuracy=accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions,average='macro').round(2)
        recall=recall_score(y_test,predictions)
        f1score=f1_score(y_test,predictions).round(2)
        ROC_AUC=roc_auc_score(y_test, y_pred_proba[:,1])
        dict_clf[clf_name]=[accuracy,precision,recall,f1score,ROC_AUC]
    df_models_scores = pd.DataFrame(dict_clf, index=['Accuracy', 'Precision', 'Recall', 'F1 Score','roc_auc_score'])
    return df_models_scores

#### Random Forest -- using  word2vec training our own data

In [12]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0,n_estimators=800,max_features='auto',max_depth=110,
                            min_samples_split=10,min_samples_leaf=1,bootstrap=False)
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [13]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect_avg)
print("Accuracy score : {}".format(accuracy_score(y_test, y_pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, y_pred)))
print("Classification Report")
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
print("Precision : {}".format(precision))
recall = recall_score(y_test, y_pred)
print("Recall : {}".format(recall))

Accuracy score : 0.8494966854898109
Confusion matrix : 
 [[2991   72]
 [ 541  469]]
Classification Report
              precision    recall  f1-score   support

           0       0.85      0.98      0.91      3063
           1       0.87      0.46      0.60      1010

    accuracy                           0.85      4073
   macro avg       0.86      0.72      0.76      4073
weighted avg       0.85      0.85      0.83      4073

Precision : 0.866913123844732
Recall : 0.4643564356435644


## Using pretrained word2vec model

In [14]:
import gensim.downloader

In [43]:
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


##### Load the  pretrained model

In [ ]:
# !wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [15]:
path_to_model = 'Embeddings//GoogleNews-vectors-negative300.bin'

In [20]:
from gensim.models.keyedvectors import KeyedVectors
%time w2v_model = KeyedVectors.load_word2vec_format(path_to_model, binary=True)
print('done loading Word2Vec')

CPU times: user 19.7 s, sys: 1.6 s, total: 21.3 s
Wall time: 21.4 s
done loading Word2Vec


In [21]:
#Inspect the model
word2vec_vocab = list(w2v_model.index_to_key)

In [22]:
word2vec_vocab_lower = [item.lower() for item in word2vec_vocab]

In [23]:
print(len(word2vec_vocab))

3000000


In [24]:
# Creating a feature vector by averaging all embeddings for all sentences
def embedding_feats(list_of_lists):
    DIMENSION = 300
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0 + 1e-5 # to avoid divide-by-zero 
        for token in tokens:
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this +=1
        if(count_for_this!=0):
            feats.append(feat_for_this/count_for_this) 
        else:
            feats.append(zero_vector)
    return feats


train_vectors = embedding_feats(X_train)
print(len(train_vectors))

16289


In [25]:
test_vectors = embedding_feats(X_test)
print(len(test_vectors))


4073


### RandomForest

In [26]:
rf = RandomForestClassifier(random_state=0,n_estimators=1000,max_features='sqrt',max_depth=100,
                            min_samples_split=2,min_samples_leaf=1,bootstrap=False)
rf_model = rf.fit(train_vectors, y_train.values.ravel())

In [27]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, y_pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, y_pred)))
print("Classification Report")
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
print("Precision : {}".format(precision))
recall = recall_score(y_test, y_pred)
print("Recall : {}".format(recall))

Accuracy score : 0.8350110483672968
Confusion matrix : 
 [[3013   50]
 [ 622  388]]
Classification Report
              precision    recall  f1-score   support

           0       0.83      0.98      0.90      3063
           1       0.89      0.38      0.54      1010

    accuracy                           0.84      4073
   macro avg       0.86      0.68      0.72      4073
weighted avg       0.84      0.84      0.81      4073

Precision : 0.8858447488584474
Recall : 0.38415841584158417


##### Hyperparameter Tuning

In [28]:
# Number of trees in random forest
n_estimators =[int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth =[int(x) for x in np.linspace(10, 110, num = 11)]
#Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
              }
pprint(random_grid)



{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}


In [29]:
rf_clf1=RandomForestClassifier(random_state=0)

In [31]:
rf_random = RandomizedSearchCV(estimator=rf_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(train_vectors, y_train.values.ravel())

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1000; total time= 4.3min
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=900; total time= 4.2min
[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=  51.3s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time= 2.4min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=500; total time= 2.2min
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=  34.1s
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100;

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, return_train_score=True, verbose=2)

In [32]:
rf_random.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 70,
 'bootstrap': False}

#### Using with Best Params 

In [36]:
rf_clf_tuned=RandomForestClassifier(random_state=0,n_estimators=400,max_features='sqrt',max_depth=70,min_samples_split=2,min_samples_leaf=1,bootstrap=False)

rf_clf_tuned.fit(train_vectors, y_train.values.ravel())

pred1 = rf_clf_tuned.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred1)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred1)))

Accuracy score : 0.833783452000982
Confusion matrix : 
 [[3008   55]
 [ 622  388]]


In [37]:
print("F1 score :")
print(f1_score(y_test, pred1))
print("Classification Report")
print(classification_report(y_test, pred1))

F1 score :
0.5340674466620785
Classification Report
              precision    recall  f1-score   support

           0       0.83      0.98      0.90      3063
           1       0.88      0.38      0.53      1010

    accuracy                           0.83      4073
   macro avg       0.85      0.68      0.72      4073
weighted avg       0.84      0.83      0.81      4073



### Support Vector Machine

In [38]:
svm_clf=SVC(random_state=0)

In [39]:
svm_clf.fit(train_vectors,y_train)

pred = svm_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.8409035109256077
Confusion matrix : 
 [[3005   58]
 [ 590  420]]
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.98      0.90      3063
           1       0.88      0.42      0.56      1010

    accuracy                           0.84      4073
   macro avg       0.86      0.70      0.73      4073
weighted avg       0.85      0.84      0.82      4073



#### Hyperparameter Tuning

In [40]:
# Create the random grid
random_grid = {
    'C': [0.1, 1, 10, 100, 1000],
               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
               'kernel':  ['rbf', 'poly', 'sigmoid'],
              }
pprint(random_grid)


{'C': [0.1, 1, 10, 100, 1000],
 'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
 'kernel': ['rbf', 'poly', 'sigmoid']}


In [41]:

svm_clf1=SVC(random_state=0)

rf_random = RandomizedSearchCV(estimator=svm_clf1, param_distributions=random_grid,
                              n_iter = 50, 
                              cv = 5, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(train_vectors,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(cv=5, estimator=SVC(random_state=0), n_iter=50, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10, 100, 1000],
                                        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                                        'kernel': ['rbf', 'poly', 'sigmoid']},
                   random_state=42, return_train_score=True, verbose=2)

[CV] END ......................C=0.1, gamma=0.1, kernel=poly; total time=  43.2s
[CV] END .......................C=10, gamma=0.1, kernel=poly; total time= 1.2min
[CV] END .......................C=1, gamma=0.01, kernel=poly; total time= 1.3min
[CV] END .....................C=10, gamma=0.001, kernel=poly; total time= 1.3min
[CV] END ...................C=100, gamma=0.0001, kernel=poly; total time= 1.3min
[CV] END ...............C=1000, gamma=0.0001, kernel=sigmoid; total time= 1.4min
[CV] END .................C=1000, gamma=0.01, kernel=sigmoid; total time= 1.4min
[CV] END .......................C=1000, gamma=1, kernel=poly; total time= 5.0min
[CV] END .....................C=10, gamma=0.001, kernel=poly; total time= 1.1min
[CV] END .....................C=100, gamma=1, kernel=sigmoid; total time= 1.3min
[CV] END ...............C=1000, gamma=0.0001, kernel=sigmoid; total time= 1.3min
[CV] END .................C=1000, gamma=0.01, kernel=sigmoid; total time= 1.5min
[CV] END ...................

[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 2.2min
[CV] END .................C=100, gamma=0.001, kernel=sigmoid; total time= 1.4min
[CV] END ..........................C=10, gamma=1, kernel=rbf; total time= 1.7min
[CV] END ..................C=0.1, gamma=0.01, kernel=sigmoid; total time= 1.1min
[CV] END ..................C=1000, gamma=0.0001, kernel=poly; total time= 1.4min
[CV] END ....................C=1000, gamma=0.001, kernel=rbf; total time= 1.6min
[CV] END .................C=100, gamma=0.001, kernel=sigmoid; total time= 1.4min
[CV] END ..........................C=10, gamma=1, kernel=rbf; total time= 1.6min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 1.8min
[CV] END ....................C=1000, gamma=0.001, kernel=rbf; total time= 1.6min
[CV] END ...................C=0.1, gamma=0.1, kernel=sigmoid; total time= 1.4min
[CV] END .................C=10, gamma=0.0001, kernel=sigmoid; total time= 1.2min
[CV] END ..................C

[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 2.2min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 1.7min
[CV] END .....................C=100, gamma=1, kernel=sigmoid; total time=  48.5s
[CV] END ......................C=1, gamma=0.001, kernel=poly; total time= 1.1min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 1.9min
[CV] END ....................C=0.1, gamma=0.001, kernel=poly; total time=  47.1s
[CV] END .......................C=10, gamma=0.1, kernel=poly; total time= 1.3min
[CV] END .......................C=1, gamma=0.01, kernel=poly; total time= 1.2min
[CV] END .....................C=10, gamma=0.001, kernel=poly; total time= 1.2min
[CV] END ...................C=100, gamma=0.0001, kernel=poly; total time= 1.1min
[CV] END ......................C=1, gamma=0.001, kernel=poly; total time= 1.2min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 2.0min
[CV] END ...................

In [42]:
rf_random.best_params_

{'kernel': 'rbf', 'gamma': 1, 'C': 10}

#### Using with Best Params 

In [89]:
svm_clf_tuned=SVC(random_state=0,kernel='rbf',gamma=1,C=10,probability=True)

svm_clf_tuned.fit(train_vectors, y_train.values.ravel())

pred1 = svm_clf_tuned.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred1)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred1)))

Accuracy score : 0.8502332433095998
Confusion matrix : 
 [[2990   73]
 [ 537  473]]


In [90]:
print("F1 score :")
print(f1_score(y_test, pred1))
print("Classification Report")
print(classification_report(y_test, pred1))

F1 score :
0.6079691516709512
Classification Report
              precision    recall  f1-score   support

           0       0.85      0.98      0.91      3063
           1       0.87      0.47      0.61      1010

    accuracy                           0.85      4073
   macro avg       0.86      0.72      0.76      4073
weighted avg       0.85      0.85      0.83      4073



## KNN

In [45]:
#Create KNN Object.
knn_clf = KNeighborsClassifier()

In [46]:
knn_clf.fit(train_vectors,y_train)

pred = knn_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.7905720599067027
Confusion matrix : 
 [[2729  334]
 [ 519  491]]
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.89      0.86      3063
           1       0.60      0.49      0.54      1010

    accuracy                           0.79      4073
   macro avg       0.72      0.69      0.70      4073
weighted avg       0.78      0.79      0.78      4073



#### Hyperparameter Tuning

In [47]:
# Create the random grid
random_grid = {
    'leaf_size': list(range(1,50)),
               'n_neighbors': list(range(1,30)),
               'p':  [1,2],
              }
pprint(random_grid)

{'leaf_size': [1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49],
 'n_neighbors': [1,
                 2,
                 3,
                 

In [48]:

knn_clf1=KNeighborsClassifier()

rf_random = RandomizedSearchCV(estimator=knn_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(train_vectors,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=KNeighborsClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...],
                                        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19, 20,
                                                        21, 22, 23, 24, 25, 26,
                                                        27, 28, 29],
                                        'p': [1, 2]},
                   random_state=42, return_train_score=True, verbose=2

In [49]:
rf_random.best_params_

{'p': 1, 'n_neighbors': 28, 'leaf_size': 7}

#### Using with Best Params 

In [50]:
knn_clf_tuned=KNeighborsClassifier(n_neighbors=28,leaf_size=7,p=1)
knn_clf_tuned.fit(train_vectors,y_train)

pred = knn_clf_tuned.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.8173336606923643
Confusion matrix : 
 [[2939  124]
 [ 620  390]]
Classification Report
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      3063
           1       0.76      0.39      0.51      1010

    accuracy                           0.82      4073
   macro avg       0.79      0.67      0.70      4073
weighted avg       0.81      0.82      0.79      4073

F1 score :
0.5118110236220473


### Xgboost

In [51]:
#Create KNN Object.
xg_clf=XGBClassifier(random_state=0)

xg_clf.fit(train_vectors,y_train)

pred = xg_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))


Accuracy score : 0.8443407807512889
Confusion matrix : 
 [[2920  143]
 [ 491  519]]
Classification Report
              precision    recall  f1-score   support

           0       0.86      0.95      0.90      3063
           1       0.78      0.51      0.62      1010

    accuracy                           0.84      4073
   macro avg       0.82      0.73      0.76      4073
weighted avg       0.84      0.84      0.83      4073



#### HYperparameter Tuning

In [52]:
# Create the random grid
random_grid = {
    'min_child_weight': range(1,6,2),
    'gamma': list(range(1,30)),
    'max_depth':  range(3,15,2),
    'alpha': [1e-5, 1e-2, 0.1,1,10,50, 100],
    'subsample':np.arange(0.5, 1.0, 0.1),
    'scale_pos_weight':range(1,5,1),
    'colsample_bytree':np.arange(0.5, 1.0, 0.1),
    'n_estimators':[100, 250, 500],
    'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
    'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
    }
pprint(random_grid)

{'alpha': [1e-05, 0.01, 0.1, 1, 10, 50, 100],
 'colsample_bylevel': array([0.5, 0.6, 0.7, 0.8, 0.9]),
 'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
 'gamma': [1,
           2,
           3,
           4,
           5,
           6,
           7,
           8,
           9,
           10,
           11,
           12,
           13,
           14,
           15,
           16,
           17,
           18,
           19,
           20,
           21,
           22,
           23,
           24,
           25,
           26,
           27,
           28,
           29],
 'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
 'max_depth': range(3, 15, 2),
 'min_child_weight': range(1, 6, 2),
 'n_estimators': [100, 250, 500],
 'scale_pos_weight': range(1, 5),
 'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])}


In [53]:

xg_clf1=XGBClassifier(random_state=0)


rf_random = RandomizedSearchCV(estimator=xg_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(train_vectors,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None,...
                                        'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'gam

In [55]:
rf_random.best_params_

{'subsample': 0.8999999999999999,
 'scale_pos_weight': 4,
 'n_estimators': 500,
 'min_child_weight': 3,
 'max_depth': 11,
 'learning_rate': 0.01,
 'gamma': 3,
 'colsample_bytree': 0.5,
 'colsample_bylevel': 0.8999999999999999,
 'alpha': 0.1}

In [56]:
xg_clf_tuned=XGBClassifier(subsample=0.89,scale_pos_weight=4,n_estimators=500,min_child_weight=3,max_depth=11,learning_rate=0.01,
                     gamma=3,colsample_bytree=0.5,colsample_bylevel=0.89,alpha=0.1)
xg_clf_tuned.fit(train_vectors,y_train)

pred = xg_clf_tuned.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.8448318192978149
Confusion matrix : 
 [[2858  205]
 [ 427  583]]
Classification Report
              precision    recall  f1-score   support

           0       0.87      0.93      0.90      3063
           1       0.74      0.58      0.65      1010

    accuracy                           0.84      4073
   macro avg       0.80      0.76      0.77      4073
weighted avg       0.84      0.84      0.84      4073

F1 score :
0.6484983314794216


### Naiva Bayes

In [65]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
nb_clf=GaussianNB()
#p = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
nb_clf.fit(train_vectors,y_train) 

pred = nb_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.7520255340044193
Confusion matrix : 
 [[3063    0]
 [1010    0]]
Classification Report
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      3063
           1       0.00      0.00      0.00      1010

    accuracy                           0.75      4073
   macro avg       0.38      0.50      0.43      4073
weighted avg       0.57      0.75      0.65      4073



#### HYperparameter Tuning

In [72]:
# Create the random grid
random_grid = {
    'var_smoothing': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]
#    # 'class_prior':[True,False],
#     'fit_prior':[True,False]
              }
pprint(random_grid)

{'var_smoothing': [1e-05, 0.0001, 0.001, 0.1, 1, 10, 100, 1000]}


In [73]:

nb_clf1=GaussianNB()


rf_random = RandomizedSearchCV(estimator=nb_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(train_vectors,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


RandomizedSearchCV(cv=3, estimator=GaussianNB(), n_iter=100, n_jobs=-1,
                   param_distributions={'var_smoothing': [1e-05, 0.0001, 0.001,
                                                          0.1, 1, 10, 100,
                                                          1000]},
                   random_state=42, return_train_score=True, verbose=2)

In [71]:
nb_clf.get_params()

{'priors': None, 'var_smoothing': 1e-09}

In [74]:
rf_random.best_params_

{'var_smoothing': 10}

In [75]:
nb_clf_tuned=GaussianNB(var_smoothing=10)

nb_clf_tuned.fit(train_vectors,y_train)

pred = nb_clf_tuned.predict(test_vectors )
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.7795236926098699
Confusion matrix : 
 [[2817  246]
 [ 652  358]]
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.92      0.86      3063
           1       0.59      0.35      0.44      1010

    accuracy                           0.78      4073
   macro avg       0.70      0.64      0.65      4073
weighted avg       0.76      0.78      0.76      4073

F1 score :
0.4436183395291202


### Logistic Regression

In [76]:
from sklearn.linear_model import LogisticRegression

In [77]:
logticreg_clf=LogisticRegression()

logticreg_clf.fit(train_vectors,y_train)

pred = logticreg_clf.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.8215074883378345
Confusion matrix : 
 [[2985   78]
 [ 649  361]]
Classification Report
              precision    recall  f1-score   support

           0       0.82      0.97      0.89      3063
           1       0.82      0.36      0.50      1010

    accuracy                           0.82      4073
   macro avg       0.82      0.67      0.69      4073
weighted avg       0.82      0.82      0.79      4073



#### HYperparameter Tuning

In [78]:
# Create the random grid
random_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
     'C':[100, 10, 1.0, 0.1, 0.01],
    'penalty':['l1','l2','elasticnet']
              }
pprint(random_grid)

{'C': [100, 10, 1.0, 0.1, 0.01],
 'penalty': ['l1', 'l2', 'elasticnet'],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}


In [79]:

logticreg_clf_tuned=LogisticRegression()


rf_random = RandomizedSearchCV(estimator=logticreg_clf_tuned, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(train_vectors,y_train)

Fitting 3 folds for each of 75 candidates, totalling 225 fits


RandomizedSearchCV(cv=3, estimator=LogisticRegression(), n_iter=100, n_jobs=-1,
                   param_distributions={'C': [100, 10, 1.0, 0.1, 0.01],
                                        'penalty': ['l1', 'l2', 'elasticnet'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']},
                   random_state=42, return_train_score=True, verbose=2)

In [80]:
rf_random.best_params_

{'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}

### Using Best Params

In [84]:
logticreg_clf_tuned=LogisticRegression(solver='lbfgs',penalty='l2',C=10)

logticreg_clf_tuned.fit(train_vectors,y_train)

pred = logticreg_clf_tuned.predict(test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.838939356739504
Confusion matrix : 
 [[2948  115]
 [ 541  469]]
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.96      0.90      3063
           1       0.80      0.46      0.59      1010

    accuracy                           0.84      4073
   macro avg       0.82      0.71      0.74      4073
weighted avg       0.83      0.84      0.82      4073

F1 score :
0.588456712672522


### Consolidating all classifiers accuracy scores

In [91]:
classifiers = [('Logistic Regression',logticreg_clf_tuned),('Naive Bayes',nb_clf_tuned),('Xgboost',xg_clf_tuned),
              ('KNN',knn_clf_tuned),('Random Forest',rf_clf_tuned),("SVM",svm_clf_tuned)]

In [92]:
df_model_scores=model_comparison_table(test_vectors,y_test,classifiers)
df_model_scores.head(20).T.sort_values(by='Accuracy', ascending=False)

,Accuracy,Precision,Recall,F1 Score,roc_auc_score
SVM,0.850233,0.86,0.468317,0.61,0.853104
Xgboost,0.844832,0.80,0.577228,0.65,0.862210
Logistic Regression,0.838939,0.82,0.464356,0.59,0.828033
Random Forest,0.833783,0.85,0.384158,0.53,0.836854
KNN,0.817334,0.79,0.386139,0.51,0.804381
Naive Bayes,0.779524,0.70,0.354455,0.44,0.744662


## Pretrained Glove

##### Download the pretrained zip file

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

#### Unzip it

In [ ]:
# !unzip glove*.zip

In [93]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [95]:
# Load 300 dimension Glove pretrained model "glove.6B.300d.txt" and convert to word2vec format",
path_to_model = '//gpfs//home//psc21zcu//FakeNews_Detection/Dissertation_Work//Embeddings//glove.6B.300d.txt'
output_file = '//gpfs//home//psc21zcu//FakeNews_Detection/Dissertation_Work//Embeddings///gensim_glove.6B.300d.txt'
glove2word2vec(path_to_model, output_file)

(400000, 300)

In [96]:
path_to_model

'//gpfs//home//psc21zcu//FakeNews_Detection/Dissertation_Work//Embeddings//glove.6B.300d.txt'

In [97]:
%time glove_model = KeyedVectors.load_word2vec_format(output_file, binary=False)
print('done loading Glove Pretrained Embedding')

CPU times: user 43.3 s, sys: 602 ms, total: 43.9 s
Wall time: 44 s
done loading Glove Pretrained Embedding


In [99]:
# https://colab.research.google.com/github/practical-nlp/practical-nlp/blob/master/Ch4/03_Word2Vec_Example.ipynb#scrollTo=m-WjFyC6b_IE

# Creating a feature vector by averaging all embeddings for all sentences
def glove_embedding_feats(list_of_lists):
    DIMENSION = 300
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0 + 1e-5 # to avoid divide-by-zero 
        for token in tokens:
            if token in glove_model:
                feat_for_this += glove_model[token]
                count_for_this +=1
        if(count_for_this!=0):
            feats.append(feat_for_this/count_for_this) 
        else:
            feats.append(zero_vector)
    return feats


glove_train_vectors = glove_embedding_feats(X_train)
print(len(glove_train_vectors))

16289


In [100]:
glove_test_vectors = glove_embedding_feats(X_test)
print(len(glove_test_vectors))

4073


### RandomForest

In [30]:
rf = RandomForestClassifier(random_state=0)
rf_model = rf.fit(glove_train_vectors, y_train.values.ravel())

In [31]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, y_pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, y_pred)))
print("Classification Report")
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
print("Precision : {}".format(precision))
recall = recall_score(y_test, y_pred)
print("Recall : {}".format(recall))

Accuracy score : 0.8273999508961454
Confusion matrix : 
 [[3014   49]
 [ 654  356]]
Classification Report
              precision    recall  f1-score   support

           0       0.82      0.98      0.90      3063
           1       0.88      0.35      0.50      1010

    accuracy                           0.83      4073
   macro avg       0.85      0.67      0.70      4073
weighted avg       0.84      0.83      0.80      4073

Precision : 0.8790123456790123
Recall : 0.35247524752475246


##### Hyperparameter Tuning

In [103]:
# Number of trees in random forest
n_estimators =[int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth =[int(x) for x in np.linspace(10, 110, num = 11)]
#Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
              }
pprint(random_grid)



{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}


In [104]:
rf_clf1=RandomForestClassifier(random_state=0)

In [105]:
rf_random = RandomizedSearchCV(estimator=rf_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(glove_train_vectors, y_train.values.ravel())

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time= 1.4min
[CV] END bootstrap=True, max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=500; total time= 1.4min
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=  29.5s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=  27.2s
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=  29.7s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=  14.5s
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=1

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1000; total time= 2.0min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time= 3.8min
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=1000; total time= 4.9min
[CV] END bootstrap=False, max_depth=70, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time= 2.3min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  57.9s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=700; total time= 3.2min
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=  12.2s
[CV] END bootstrap=True, max_depth=60, m

[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time= 1.4min
[CV] END bootstrap=True, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1000; total time= 3.1min
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=900; total time= 4.2min
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time= 2.1min
[CV] END bootstrap=True, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=900; total time= 2.3min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=500; total time= 1.4min
[CV] END bootstrap=True, max_depth=70, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=  33.7s
[CV] END bootstrap=True, max_depth=90, max_

[CV] END bootstrap=True, max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=900; total time= 2.5min
[CV] END bootstrap=False, max_depth=70, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1000; total time= 4.6min
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 1.6min
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time= 2.8min
[CV] END bootstrap=True, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=  18.1s
[CV] END bootstrap=True, max_depth=40, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=600; total time= 1.6min
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  15.4s
[CV] END bootstrap=False, max_depth=40, ma

[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=1000; total time= 4.4min
[CV] END bootstrap=True, max_depth=40, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=900; total time= 2.7min
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time= 3.5min
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=900; total time= 3.5min
[CV] END bootstrap=True, max_depth=110, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=700; total time= 1.9min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time= 3.8min
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=800; total time= 2.1min
[CV] END bootstrap=False, max_depth=80

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, return_train_score=True, verbose=2)

In [107]:
rf_random.best_params_

{'n_estimators': 900,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 20,
 'bootstrap': False}

#### Using with Best Params 

In [109]:
rf_clf_tuned_g=RandomForestClassifier(random_state=0,n_estimators=900,max_features='auto',max_depth=20,min_samples_split=5,min_samples_leaf=1,bootstrap=False)

rf_clf_tuned_g.fit(glove_train_vectors, y_train.values.ravel())

pred1 = rf_clf_tuned_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred1)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred1)))

Accuracy score : 0.8266633930763565
Confusion matrix : 
 [[3020   43]
 [ 663  347]]


In [110]:
print("F1 score :")
print(f1_score(y_test, pred1))
print("Classification Report")
print(classification_report(y_test, pred1))

F1 score :
0.49571428571428566
Classification Report
              precision    recall  f1-score   support

           0       0.82      0.99      0.90      3063
           1       0.89      0.34      0.50      1010

    accuracy                           0.83      4073
   macro avg       0.85      0.66      0.70      4073
weighted avg       0.84      0.83      0.80      4073



### Support Vector Machine

In [111]:
svm_clf=SVC(random_state=0,probability=True)

In [112]:
svm_clf.fit(glove_train_vectors,y_train)

pred = svm_clf.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.8185612570586791
Confusion matrix : 
 [[3018   45]
 [ 694  316]]
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.99      0.89      3063
           1       0.88      0.31      0.46      1010

    accuracy                           0.82      4073
   macro avg       0.84      0.65      0.68      4073
weighted avg       0.83      0.82      0.78      4073



#### Hyperparameter Tuning

In [113]:
# Create the random grid
random_grid = {
    'C': [0.1, 1, 10, 100, 1000],
               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
               'kernel':  ['rbf', 'poly', 'sigmoid'],
              }
pprint(random_grid)


{'C': [0.1, 1, 10, 100, 1000],
 'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
 'kernel': ['rbf', 'poly', 'sigmoid']}


In [114]:

svm_clf1=SVC(random_state=0)

rf_random = RandomizedSearchCV(estimator=svm_clf1, param_distributions=random_grid,
                              n_iter = 50, 
                              cv = 5, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(glove_train_vectors,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END ......................C=0.1, gamma=0.1, kernel=poly; total time=  45.8s
[CV] END .....................C=10, gamma=0.0001, kernel=rbf; total time= 2.1min
[CV] END ......................C=10, gamma=0.001, kernel=rbf; total time= 1.5min
[CV] END ..................C=100, gamma=0.01, kernel=sigmoid; total time= 1.1min
[CV] END .....................C=100, gamma=0.001, kernel=rbf; total time= 1.7min
[CV] END .....................C=1, gamma=0.0001, kernel=poly; total time=  59.8s
[CV] END ......................C=100, gamma=0.1, kernel=poly; total time= 1.8min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 1.8min
[CV] END .....................C=100, gamma=1, kernel=sigmoid; total time=  58.4s
[CV] END ....................C=1000, gamma=1, kernel=sigmoid; total time=  55.5s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 1.7min
[CV] END ....................C=1000, gamma=0.00

[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 1.5min
[CV] END ...................C=0.1, gamma=0.1, kernel=sigmoid; total time= 1.2min
[CV] END ..........................C=1, gamma=1, kernel=poly; total time= 4.0min
[CV] END ...................C=0.1, gamma=0.0001, kernel=poly; total time= 1.3min
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time= 1.8min
[CV] END .......................C=1000, gamma=1, kernel=poly; total time=10.1min
[CV] END ...............C=1000, gamma=0.0001, kernel=sigmoid; total time= 1.5min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 1.6min
[CV] END ......................C=1000, gamma=0.1, kernel=rbf; total time= 3.8min
[CV] END ..........................C=1, gamma=1, kernel=poly; total time= 4.0min
[CV] END ...............C=1000, gamma=0.0001, kernel=sigmoid; total time= 1.3min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 1.8min
[CV] END ...................

RandomizedSearchCV(cv=5, estimator=SVC(random_state=0), n_iter=50, n_jobs=-1,
                   param_distributions={'C': [0.1, 1, 10, 100, 1000],
                                        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                                        'kernel': ['rbf', 'poly', 'sigmoid']},
                   random_state=42, return_train_score=True, verbose=2)

In [116]:
rf_random.best_params_

{'kernel': 'rbf', 'gamma': 1, 'C': 10}

#### Using with Best Params 

In [117]:
svm_clf_tuned_g=SVC(random_state=0,kernel='rbf',gamma=1,C=10,probability=True)

svm_clf_tuned_g.fit(glove_train_vectors, y_train.values.ravel())

pred1 = svm_clf_tuned_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred1)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred1)))

[CV] END ....................C=10, gamma=0.1, kernel=sigmoid; total time=  42.7s
[CV] END .......................C=10, gamma=0.1, kernel=poly; total time= 1.1min
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 1.8min
[CV] END ..........................C=10, gamma=1, kernel=rbf; total time= 2.5min
[CV] END ...................C=10, gamma=0.01, kernel=sigmoid; total time= 1.3min
[CV] END ........................C=100, gamma=1, kernel=poly; total time= 5.8min
[CV] END ....................C=0.1, gamma=0.0001, kernel=rbf; total time= 1.5min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 1.8min
[CV] END .....................C=100, gamma=1, kernel=sigmoid; total time=  57.7s
[CV] END ........................C=1, gamma=0.1, kernel=poly; total time= 1.2min
[CV] END .......................C=1, gamma=1, kernel=sigmoid; total time= 1.6min
[CV] END ........................C=100, gamma=1, kernel=poly; total time= 6.0min
[CV] END ...................

In [118]:
print("F1 score :")
print(f1_score(y_test, pred1))
print("Classification Report")
print(classification_report(y_test, pred1))

F1 score :
0.657191586128482
Classification Report
              precision    recall  f1-score   support

           0       0.87      0.94      0.91      3063
           1       0.77      0.57      0.66      1010

    accuracy                           0.85      4073
   macro avg       0.82      0.76      0.78      4073
weighted avg       0.85      0.85      0.84      4073



## KNN

In [119]:
#Create KNN Object.
knn_clf = KNeighborsClassifier()

In [120]:
knn_clf.fit(glove_train_vectors,y_train)

pred = knn_clf.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.8023569850233243
Confusion matrix : 
 [[2796  267]
 [ 538  472]]
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.91      0.87      3063
           1       0.64      0.47      0.54      1010

    accuracy                           0.80      4073
   macro avg       0.74      0.69      0.71      4073
weighted avg       0.79      0.80      0.79      4073



#### Hyperparameter Tuning

In [121]:
# Create the random grid
random_grid = {
    'leaf_size': list(range(1,50)),
               'n_neighbors': list(range(1,30)),
               'p':  [1,2],
              }
pprint(random_grid)

{'leaf_size': [1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49],
 'n_neighbors': [1,
                 2,
                 3,
                 

In [122]:

knn_clf1=KNeighborsClassifier()

rf_random = RandomizedSearchCV(estimator=knn_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(glove_train_vectors,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=KNeighborsClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...],
                                        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19, 20,
                                                        21, 22, 23, 24, 25, 26,
                                                        27, 28, 29],
                                        'p': [1, 2]},
                   random_state=42, return_train_score=True, verbose=2

In [123]:
rf_random.best_params_

{'p': 1, 'n_neighbors': 27, 'leaf_size': 13}

#### Using with Best Params 

In [124]:
knn_clf_tuned_g=KNeighborsClassifier(n_neighbors=27,leaf_size=13,p=1)
knn_clf_tuned_g.fit(glove_train_vectors,y_train)

pred = knn_clf_tuned_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.8178246992388902
Confusion matrix : 
 [[2945  118]
 [ 624  386]]
Classification Report
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      3063
           1       0.77      0.38      0.51      1010

    accuracy                           0.82      4073
   macro avg       0.80      0.67      0.70      4073
weighted avg       0.81      0.82      0.79      4073

F1 score :
0.5099075297225892


In [139]:
classifiers = [('knn',knn_clf_tuned_g)]
df_model_scores=model_comparison_table(glove_test_vectors,y_test,classifiers)
df_model_scores.head(20).T

,Accuracy,Precision,Recall,F1 Score,roc_auc_score
knn,0.817825,0.8,0.382178,0.51,0.811306


### Xgboost

In [140]:
#Create KNN Object.
xg_clf=XGBClassifier(random_state=0)

xg_clf.fit(glove_train_vectors,y_train)

pred = xg_clf.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))


Accuracy score : 0.844095261478026
Confusion matrix : 
 [[2928  135]
 [ 500  510]]
Classification Report
              precision    recall  f1-score   support

           0       0.85      0.96      0.90      3063
           1       0.79      0.50      0.62      1010

    accuracy                           0.84      4073
   macro avg       0.82      0.73      0.76      4073
weighted avg       0.84      0.84      0.83      4073



#### HYperparameter Tuning

In [141]:
# Create the random grid
random_grid = {
    'min_child_weight': range(1,6,2),
    'gamma': list(range(1,30)),
    'max_depth':  range(3,15,2),
    'alpha': [1e-5, 1e-2, 0.1,1,10,50, 100],
    'subsample':np.arange(0.5, 1.0, 0.1),
    'scale_pos_weight':range(1,5,1),
    'colsample_bytree':np.arange(0.5, 1.0, 0.1),
    'n_estimators':[100, 250, 500],
    'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
    'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
    }
pprint(random_grid)

{'alpha': [1e-05, 0.01, 0.1, 1, 10, 50, 100],
 'colsample_bylevel': array([0.5, 0.6, 0.7, 0.8, 0.9]),
 'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
 'gamma': [1,
           2,
           3,
           4,
           5,
           6,
           7,
           8,
           9,
           10,
           11,
           12,
           13,
           14,
           15,
           16,
           17,
           18,
           19,
           20,
           21,
           22,
           23,
           24,
           25,
           26,
           27,
           28,
           29],
 'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
 'max_depth': range(3, 15, 2),
 'min_child_weight': range(1, 6, 2),
 'n_estimators': [100, 250, 500],
 'scale_pos_weight': range(1, 5),
 'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])}


In [142]:

xg_clf1=XGBClassifier(random_state=0)


rf_random = RandomizedSearchCV(estimator=xg_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(glove_train_vectors,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None,...
                                        'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'gam

In [143]:
rf_random.best_params_

{'subsample': 0.7,
 'scale_pos_weight': 2,
 'n_estimators': 500,
 'min_child_weight': 1,
 'max_depth': 5,
 'learning_rate': 0.1,
 'gamma': 3,
 'colsample_bytree': 0.7999999999999999,
 'colsample_bylevel': 0.8999999999999999,
 'alpha': 1}

In [144]:
xg_clf_tuned_g=XGBClassifier(subsample=0.7,scale_pos_weight=2,n_estimators=500,min_child_weight=1,max_depth=5,learning_rate=0.1,
                     gamma=3,colsample_bytree=0.79,colsample_bylevel=0.89,alpha=1)
xg_clf_tuned_g.fit(glove_train_vectors,y_train)

pred = xg_clf_tuned_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

[CV] END alpha=50, colsample_bylevel=0.6, colsample_bytree=0.8999999999999999, gamma=19, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=100, scale_pos_weight=2, subsample=0.7999999999999999; total time=  27.1s
[CV] END alpha=1, colsample_bylevel=0.7, colsample_bytree=0.8999999999999999, gamma=3, learning_rate=0.4, max_depth=11, min_child_weight=5, n_estimators=100, scale_pos_weight=2, subsample=0.7; total time=  55.1s
[CV] END alpha=0.01, colsample_bylevel=0.7, colsample_bytree=0.6, gamma=20, learning_rate=0.2, max_depth=9, min_child_weight=5, n_estimators=500, scale_pos_weight=2, subsample=0.6; total time= 2.8min
[CV] END alpha=1, colsample_bylevel=0.6, colsample_bytree=0.6, gamma=29, learning_rate=0.3, max_depth=7, min_child_weight=1, n_estimators=250, scale_pos_weight=1, subsample=0.5; total time=  58.2s
[CV] END alpha=50, colsample_bylevel=0.7999999999999999, colsample_bytree=0.6, gamma=21, learning_rate=0.3, max_depth=3, min_child_weight=3, n_estimators=250, sca

[CV] END alpha=0.1, colsample_bylevel=0.7, colsample_bytree=0.6, gamma=15, learning_rate=0.01, max_depth=5, min_child_weight=5, n_estimators=100, scale_pos_weight=4, subsample=0.8999999999999999; total time=  26.6s
[CV] END alpha=0.01, colsample_bylevel=0.7, colsample_bytree=0.7999999999999999, gamma=13, learning_rate=0.3, max_depth=13, min_child_weight=5, n_estimators=100, scale_pos_weight=4, subsample=0.7; total time= 1.0min
[CV] END alpha=50, colsample_bylevel=0.7, colsample_bytree=0.7, gamma=9, learning_rate=0.1, max_depth=3, min_child_weight=3, n_estimators=100, scale_pos_weight=4, subsample=0.5; total time=  15.1s
[CV] END alpha=100, colsample_bylevel=0.8999999999999999, colsample_bytree=0.5, gamma=9, learning_rate=0.1, max_depth=7, min_child_weight=5, n_estimators=100, scale_pos_weight=2, subsample=0.6; total time=  26.5s
[CV] END alpha=10, colsample_bylevel=0.7999999999999999, colsample_bytree=0.8999999999999999, gamma=18, learning_rate=0.4, max_depth=3, min_child_weight=1, n_e

[CV] END alpha=50, colsample_bylevel=0.8999999999999999, colsample_bytree=0.7999999999999999, gamma=28, learning_rate=0.4, max_depth=13, min_child_weight=1, n_estimators=250, scale_pos_weight=3, subsample=0.5; total time= 2.3min
[CV] END alpha=10, colsample_bylevel=0.6, colsample_bytree=0.7, gamma=20, learning_rate=0.2, max_depth=13, min_child_weight=1, n_estimators=100, scale_pos_weight=3, subsample=0.5; total time=  46.5s
[CV] END alpha=0.1, colsample_bylevel=0.7999999999999999, colsample_bytree=0.8999999999999999, gamma=29, learning_rate=0.2, max_depth=5, min_child_weight=3, n_estimators=100, scale_pos_weight=3, subsample=0.8999999999999999; total time=  38.9s
[CV] END alpha=10, colsample_bylevel=0.6, colsample_bytree=0.7, gamma=17, learning_rate=0.2, max_depth=9, min_child_weight=3, n_estimators=100, scale_pos_weight=1, subsample=0.8999999999999999; total time=  46.3s
[CV] END alpha=0.01, colsample_bylevel=0.5, colsample_bytree=0.6, gamma=16, learning_rate=0.2, max_depth=7, min_chi

### Naiva Bayes

In [145]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
nb_clf=GaussianNB()
#p = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
nb_clf.fit(glove_train_vectors,y_train) 

pred = nb_clf.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.7520255340044193
Confusion matrix : 
 [[3063    0]
 [1010    0]]
Classification Report
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      3063
           1       0.00      0.00      0.00      1010

    accuracy                           0.75      4073
   macro avg       0.38      0.50      0.43      4073
weighted avg       0.57      0.75      0.65      4073



#### HYperparameter Tuning

In [157]:
# Create the random grid
random_grid = {
    'var_smoothing': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]
#    # 'class_prior':[True,False],
#     'fit_prior':[True,False]
              }
pprint(random_grid)

{'var_smoothing': [1e-05, 0.0001, 0.001, 0.1, 1, 10, 100, 1000]}


In [158]:

nb_clf1=GaussianNB()


rf_random = RandomizedSearchCV(estimator=nb_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(glove_train_vectors,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


RandomizedSearchCV(cv=3, estimator=GaussianNB(), n_iter=100, n_jobs=-1,
                   param_distributions={'var_smoothing': [1e-05, 0.0001, 0.001,
                                                          0.1, 1, 10, 100,
                                                          1000]},
                   random_state=42, return_train_score=True, verbose=2)

In [155]:
nb_clf.get_params()

{'priors': None, 'var_smoothing': 1e-09}

In [159]:
rf_random.best_params_

{'var_smoothing': 10}

In [163]:
nb_clf_tuned_g=GaussianNB(var_smoothing=10)

nb_clf_tuned_g.fit(glove_train_vectors,y_train)

pred = nb_clf_tuned_g.predict(glove_test_vectors )
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.7520255340044193
Confusion matrix : 
 [[3063    0]
 [1010    0]]
Classification Report
              precision    recall  f1-score   support

           0       0.75      1.00      0.86      3063
           1       0.00      0.00      0.00      1010

    accuracy                           0.75      4073
   macro avg       0.38      0.50      0.43      4073
weighted avg       0.57      0.75      0.65      4073

F1 score :
0.0


### Logistic Regression

In [164]:
from sklearn.linear_model import LogisticRegression

In [165]:
logticreg_clf=LogisticRegression()

logticreg_clf.fit(glove_train_vectors,y_train)

pred = logticreg_clf.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.8251902774367788
Confusion matrix : 
 [[2947  116]
 [ 596  414]]
Classification Report
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      3063
           1       0.78      0.41      0.54      1010

    accuracy                           0.83      4073
   macro avg       0.81      0.69      0.71      4073
weighted avg       0.82      0.83      0.80      4073



#### HYperparameter Tuning

In [166]:
# Create the random grid
random_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
     'C':[100, 10, 1.0, 0.1, 0.01],
    'penalty':['l1','l2','elasticnet']
              }
pprint(random_grid)

{'C': [100, 10, 1.0, 0.1, 0.01],
 'penalty': ['l1', 'l2', 'elasticnet'],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}


In [167]:

logticreg_clf_tuned=LogisticRegression()


rf_random = RandomizedSearchCV(estimator=logticreg_clf_tuned, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(glove_train_vectors,y_train)

Fitting 3 folds for each of 75 candidates, totalling 225 fits


RandomizedSearchCV(cv=3, estimator=LogisticRegression(), n_iter=100, n_jobs=-1,
                   param_distributions={'C': [100, 10, 1.0, 0.1, 0.01],
                                        'penalty': ['l1', 'l2', 'elasticnet'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']},
                   random_state=42, return_train_score=True, verbose=2)

In [168]:
rf_random.best_params_

{'solver': 'liblinear', 'penalty': 'l1', 'C': 10}

### Using Best Params

In [170]:
logticreg_clf_tuned_g=LogisticRegression(solver='liblinear',penalty='l1',C=10)

logticreg_clf_tuned_g.fit(glove_train_vectors,y_train)

pred = logticreg_clf_tuned_g.predict(glove_test_vectors)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.8256813159833047
Confusion matrix : 
 [[2902  161]
 [ 549  461]]
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      3063
           1       0.74      0.46      0.56      1010

    accuracy                           0.83      4073
   macro avg       0.79      0.70      0.73      4073
weighted avg       0.82      0.83      0.81      4073

F1 score :
0.564950980392157


### Consolidating all classifiers accuracy scores

In [171]:
classifiers = [('Logistic Regression',logticreg_clf_tuned_g),('Naive Bayes',nb_clf_tuned_g),('Xgboost',xg_clf_tuned_g),
              ('KNN',knn_clf_tuned_g),('Random Forest',rf_clf_tuned_g),("SVM",svm_clf_tuned_g)]

In [173]:
df_model_scores=model_comparison_table(glove_test_vectors,y_test,classifiers)
df_model_scores.head(20).T.sort_values(by='Accuracy', ascending=False)

,Accuracy,Precision,Recall,F1 Score,roc_auc_score
Xgboost,0.852443,0.82,0.591089,0.67,0.867570
SVM,0.851952,0.82,0.572277,0.66,0.864249
Random Forest,0.826663,0.85,0.343564,0.50,0.839567
Logistic Regression,0.825681,0.79,0.456436,0.56,0.823835
KNN,0.817825,0.80,0.382178,0.51,0.811306
Naive Bayes,0.752026,0.38,0.000000,0.00,0.697570
